In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

import time

import pandas as pd

c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

In [2]:
# URL base
url = "https://www.tesla.com"

postal_zips = {
    "spain": {
        'cp': '45950',
        'url': 'es_ES'
    },
    "portugal": {
        'cp': '1000-004',
        'url': 'pt_PT'
    },
    "france": {
        'cp': '70123',
        'url': 'fr_FR'
    },
    "italy": {
        'cp': '10024',
        'url': 'it_IT'
    }
}



# Configuración del diccionario de modelos
cars_types = {
    'Model S': {
        'model': 'ms',
        # 'colors': ['BLACK','WHITE','BLUE','SILVER']
    },
    'Model 3': {
        'model': 'm3',
        # 'colors': ['BLACK','WHITE','BLUE','GREY','SILVER','CHERRY']
    },
    'Model X': {
        'model': 'mx',
        # 'colors': ['BLACK','WHITE','BLUE','GREY','SILVER','CHERRY']
    },
    'Model Y': {
        'model': 'my',
        # 'colors': ['BLACK','WHITE','BLUE','SILVER','CHERRY']
    }
}


types = {
    'Nuevo': 'new',
    'Ocasion': 'used'
}


In [14]:
# Inicializar datos
data = []

# Recorrer los parámetros
for country in postal_zips:
    for type in types:
        for car_type in cars_types:
            current_car = cars_types[car_type]['model']
            current_type = types[type]
            current_country = postal_zips[country]

            # Construir URL de inventario inicial
            params = 'arrangeby=plh&range=0'
            c_driver.get(f'{url}/{current_country["url"]}/inventory/{current_type}/{current_car}?{params}')
            time.sleep(5)

            try:
                # Buscar los colores disponibles
                xpath_colors = '/html/body/div[1]/div/div[2]/aside/form/div/div[7]/fieldset/div/div'
                colors_types = c_driver.find_elements(By.XPATH, xpath_colors)
                #colors_types = c_driver.find_elements(By.XPATH, '/html/body/div[1]/div/div[1]/aside/form/div/div[7]/fieldset/div/div')
                colors = []
                for color_type in colors_types:
                    color_input = color_type.find_element(By.XPATH, './div/input')
                    color = color_input.get_attribute('id').split('_')[-1]
                    colors.append(color)

                # Recorrer los colores
                for color in colors:
                    params = f'PAINT={color}&arrangeby=plh&range=0&zip={postal_zips[country]["cp"]}'
                    c_driver.get(f'{url}/{current_country["url"]}/inventory/{current_type}/{current_car}?{params}')
                    time.sleep(5)

                    # Buscar los coches en la página tras la recarga
                    xpath = '/html/body/div[1]/div/div[2]/main/div/article'
                    #cars = c_driver.find_elements(By.XPATH, '/html/body/div[1]/div/div[1]/main/div/article')
                    cars = c_driver.find_elements(By.XPATH, xpath)
                    print(cars)
                    for car in cars:
                        try:
                            # Extraer datos del coche
                            traction = car.find_element(By.XPATH, './section[1]/div[1]/div[1]').text

                            if current_type == 'new':
                                price = car.find_element(By.XPATH, './section[1]/div[2]/div[1]/div/span[1]').text
                                year = datetime.now().year
                            else:
                                price = car.find_element(By.XPATH, './section[1]/div[2]/div/div/span').text
                                year = car.find_element(By.XPATH, './section[1]/div[1]/h3/span[1]').text

                            try:
                                km = car.find_element(By.XPATH, './section[1]/div[1]/div[2]/span').text or 0
                            except NoSuchElementException:
                                km = 0

                            range_ = car.find_element(By.XPATH, './section[5]/div/ul/li[1]/div[1]/span[1]').text
                            max_vel = car.find_element(By.XPATH, './section[5]/div/ul/li[2]/div[1]/span[1]').text
                            zero_hundred = car.find_element(By.XPATH, './section[5]/div/ul/li[3]/div[1]/span[1]').text

                            # Agregar datos al diccionario
                            data.append({
                                'sell_type': type,
                                'car_type': car_type,
                                'year': year,
                                'color': color,
                                'country': country,
                                'traction': traction,
                                'price': price,
                                'km': km,
                                'range': range_,
                                'max_vel': max_vel,
                                'zero_hundred': zero_hundred
                            })

                        except Exception as e:
                            print(f"Error procesando coche: {e}")

            except Exception as e:
                print(f"Error procesando página: {e}")

[<selenium.webdriver.remote.webelement.WebElement (session="717b88548114390f4e233208033195b0", element="f.CC7340734071EB93A96B6D02374CF60E.d.55A30169529FFC8988F69C932C6991D5.e.23655")>, <selenium.webdriver.remote.webelement.WebElement (session="717b88548114390f4e233208033195b0", element="f.CC7340734071EB93A96B6D02374CF60E.d.55A30169529FFC8988F69C932C6991D5.e.23656")>, <selenium.webdriver.remote.webelement.WebElement (session="717b88548114390f4e233208033195b0", element="f.CC7340734071EB93A96B6D02374CF60E.d.55A30169529FFC8988F69C932C6991D5.e.23657")>]
[<selenium.webdriver.remote.webelement.WebElement (session="717b88548114390f4e233208033195b0", element="f.CC7340734071EB93A96B6D02374CF60E.d.13F43DC16D10F2ADC8F3B50687121870.e.23743")>, <selenium.webdriver.remote.webelement.WebElement (session="717b88548114390f4e233208033195b0", element="f.CC7340734071EB93A96B6D02374CF60E.d.13F43DC16D10F2ADC8F3B50687121870.e.23744")>]
[<selenium.webdriver.remote.webelement.WebElement (session="717b885481143

In [15]:
df = pd.DataFrame(data)
df

,sell_type,car_type,year,color,country,traction,price,km,range,max_vel,zero_hundred
0,Nuevo,Model S,2025,WHITE,spain,"Model S, tracción a las cuatro ruedas",99.600 €,Cuentakilómetros con 503 km,634,250,"3,2"
1,Nuevo,Model S,2025,WHITE,spain,"Model S, tracción a las cuatro ruedas",102.270 €,Cuentakilómetros con 99 km,634,250,"3,2"
2,Nuevo,Model S,2025,WHITE,spain,"Model S, tracción a las cuatro ruedas",102.270 €,0,634,250,"3,2"
3,Nuevo,Model S,2025,BLACK,spain,"Model S, tracción a las cuatro ruedas",90.130 €,Cuentakilómetros con 662 km,634,250,"3,2"
4,Nuevo,Model S,2025,BLACK,spain,Plaid,100.620 €,Cuentakilómetros con 3652 km,600,322,"2,1"
...,...,...,...,...,...,...,...,...,...,...,...
1119,Ocasion,Model Y,2022,WHITE,italy,Performance a trazione integrale,45.500 €,69.439 km sul contachilometri,514,250,"3,7"
1120,Ocasion,Model Y,2023,BLACK,italy,Long Range a trazione integrale,42.000 €,82.410 km sul contachilometri,533,217,5
1121,Ocasion,Model Y,2022,BLACK,italy,Performance a trazione integrale,43.000 €,92.596 km sul contachilometri,514,250,"3,7"
1122,Ocasion,Model Y,2022,GREY,italy,Long Range a trazione integrale,41.000 €,48.960 km sul contachilometri,533,217,5


In [16]:
df.dtypes

sell_type       object
car_type        object
year            object
color           object
country         object
traction        object
price           object
km              object
range           object
max_vel         object
zero_hundred    object
dtype: object

In [17]:
df.isna().sum()

sell_type       0
car_type        0
year            0
color           0
country         0
traction        0
price           0
km              0
range           0
max_vel         0
zero_hundred    0
dtype: int64

In [18]:
df.to_csv('../../../data/tesla.com.csv')

In [10]:
# 26 / 12 / 2024

In [19]:
df.to_csv('../../../data/new_tesla.com.csv')

In [27]:
old = pd.read_csv('../../../data/tesla.com.csv')
old.drop(columns=['Unnamed: 0'], inplace=True)
full_tesla_df = pd.concat([old, df], ignore_index=True)

In [28]:
full_tesla_df.to_csv('../../../data/full_tesla_df.com.csv')